# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests 

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio' #puxando a URL da rede 
api = requests.get(url) #status de verificação se o servidor esta respondendo, 200 é bom 
api.status_code

data_json = api.json()
df = pd.DataFrame(data_json['items'])
df.head()


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,66452,Município,Barra do Piraí,3300308,RJ,PVL02.002586/2023-21,Deferido,17944.104168/2023-31,2023-09-11T14:34:27Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,30000000.00,0,1,13/09/2023
1,31436,Município,Nilópolis,3303203,RJ,PVL02.000708/2019-69,Deferido (PVL-IF),None,None,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4745738.61,0,0,21/06/2019
2,31378,Município,Petrópolis,3303906,RJ,PVL02.000583/2019-77,Deferido,17944.101202/2019-39,2019-04-11T19:29:03Z,Operação contratual interna,PNAFM,Instituição Financeira Nacional,Caixa Econômica Federal,Real,9200000.00,1,1,23/04/2019
3,62993,Município,Paraíba do Sul,3303708,RJ,PVL02.000025/2023-98,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3900000.00,0,1,28/03/2023
4,60039,Município,Petrópolis,3303906,RJ,PVL02.005942/2022-88,Deferido,17944.102384/2022-61,2022-06-21T13:12:44Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,20000000.00,0,1,23/06/2022


In [3]:
# 2) Seu código aqui

df['status'].value_counts() #filtrando a coluna 'status' e contando os maiores 

status
Deferido                                                 106
Arquivado                                                 53
Regularizado                                              26
Deferido (PVL-IF)                                         25
Arquivado por decurso de prazo                            24
Encaminhado à PGFN com manifestação técnica favorável     16
Arquivado a pedido                                         8
Em retificação pelo interessado                            4
Assinado pelo interessado (retificação)                    2
Arquivado a pedido (PVL-IF)                                1
Arquivado pela STN                                         1
Em análise                                                 1
Name: count, dtype: int64

In [21]:
# 3) Seu código aqui

df_data = df['data_status'].astype(str) #convertendo object em str 
df['ano'] = df_data.str[:4] #recortando o ano 
status = df[['status', 'ano']].value_counts() #criando uma variavel com data e status da solicitação 
status

status                          ano 
Arquivado                       2008    28
Deferido                        2007    13
                                2014    13
                                2023    12
Arquivado                       2010    11
                                        ..
Arquivado por decurso de prazo  2020     1
                                2023     1
Deferido                        2003     1
                                2004     1
Regularizado                    2023     1
Name: count, Length: 74, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [66]:
#1) Seu código aqui

url_sp = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=SP&tipo_interessado=Estado' #url com o estado de SP 
api_sp = requests.get(url_sp) #pegando informação da URL

url_rj = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado' #url com o estado de RJ 
api_rj = requests.get(url_rj) #pegando informação da URL

url_mg = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=MG&tipo_interessado=Estado'
api_mg = requests.get(url_mg)

data_sp = api_sp.json() #convertendo os dados em json 
data_rj = api_rj.json() #convertendo os dados em json
data_mg = api_mg.json()

api_estados = data_sp['items'] + data_rj['items'] + data_mg['items'] #juntando os Json dos estados estudados nessa atividade 

df_data = pd.DataFrame(api_estados) # convertendo o Json em DataFrame
df_data

def filtro_estado(df_data, estado): #criando função para localizar o estado no DataFrame com 2 parametros 
    df_filtrado = df_data.loc[df_data['uf'] == estado]
    return df_filtrado

filtro_estado(df_data, 'RJ').head() #df_data + sigla do estado requerido
    

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
97,5516,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000131/2011-09,2011-06-30T03:00:00Z,Operação contratual externa (com garantia da U...,Desenvolvimento sustentável,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.850000e+08,1,0,04/07/2011
98,7255,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000532/2012-31,2012-05-24T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.645563e+09,1,0,29/05/2012
99,13560,Estado,Rio de Janeiro,33,RJ,None,Deferido,19405.000026/2002-01,2002-06-25T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,4.994200e+04,1,0,27/06/2002
100,10561,Estado,Rio de Janeiro,33,RJ,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001228/2011-21,2012-04-18T03:00:00Z,Operação contratual interna (com garantia da U...,PAC 2 - Saneamento,Instituição Financeira Nacional,Caixa Econômica Federal,Real,4.153722e+08,1,0,02/07/2012
101,7525,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.000599/2009-71,2009-06-17T03:00:00Z,Operação contratual interna,PAC - Saneamento - 2008,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.580000e+08,1,0,18/06/2009


In [81]:
# 2) Seu código aqui

df_mg = filtro_estado(df_data, 'MG') #filtrando o estado de MG

contagem_status = df_mg['status'].value_counts() #contato dos os status e guardando em uma variavél 
arquivo_contagem = contagem_status['Arquivado por decurso de prazo'] #separando somente o status requerido
arquivo_contagem

1

In [105]:
# 3) Seu código aqui

url_ba = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=BA&tipo_interessado=Munic%C3%ADpio' #url da Baiha com municipios
api_ba = requests.get(url_ba) 
data_ba = api_ba.json()

df_ba = pd.DataFrame(data_ba['items'])

status_ba = df_ba[df_ba['status']=='Deferido'] #separando o status deferido

top_municipio_deferido = status_ba['interessado'].value_counts() #separando e contando os municipios deferidos
top_municipio_deferido

interessado
Luís Eduardo Magalhães    16
Lauro de Freitas          12
Vitória da Conquista      12
Camaçari                  10
Barreiras                 10
                          ..
Coronel João Sá            1
Feira da Mata              1
Ruy Barbosa                1
Sebastião Laranjeiras      1
Itapicuru                  1
Name: count, Length: 185, dtype: int64

In [110]:
# 4) Seu código aqui

df_ba_filtrado = df_ba[['interessado', 'status']].value_counts()
df_ba_filtrado.to_csv('df_ba_filtrado.csv')


interessado             status                        
Luís Eduardo Magalhães  Deferido                          16
Lauro de Freitas        Deferido                          12
Vitória da Conquista    Deferido                          12
Camaçari                Deferido                          10
Barreiras               Deferido                          10
                                                          ..
Lajedinho               Arquivado a pedido                 1
Laje                    Arquivado                          1
Lagoa Real              Arquivado por decurso de prazo     1
                        Arquivado                          1
Abaíra                  Arquivado                          1
Name: count, Length: 611, dtype: int64